# Data Prediction

#### Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import models
import numpy as np
from PIL import Image
import cv2
import imutils

#### Global Variables

In [2]:
bg = None
temp_image = 'temp.png'

### Resize Image

In [3]:
def resizeImage(imageName):
    img = Image.open(imageName)
    img = img.resize((100, 89), Image.ANTIALIAS)
    img.save(imageName)

## Running Average

In [4]:
def run_avg(img, weight):
    global bg
    if bg is None:
        bg = img.copy().astype("float")
        return
    cv2.accumulateWeighted(img, bg, weight)

## Segmentation

In [5]:
def segment(image):
    global bg
    sub_image = cv2.absdiff(bg.astype("uint8"), image)

    thresholded = cv2.threshold(sub_image,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)[1]

    (contor, hier) = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    if len(contor) == 0:
        return
    else:
        segmented = max(contor, key=cv2.contourArea)
        return (thresholded, segmented)

## Getting Predicted Class

In [6]:
def getPredictedClass():
    image = cv2.imread(temp_image)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_image = gray_image.reshape(89, 100, 1)
    gray_image = np.array([gray_image])
    prediction = model.predict(gray_image)
    return prediction

## Displaying Result

In [7]:
def showStatistics(prediction,textImage):
    gestures = ['fist', 'palm', 'swing']
    n = len(gestures)
    x = 30
    y =  30
    height = (n+3)*y
    width = 500
        
    predicted_gesture = gestures[np.argmax(prediction)]
    sum = 0.00
    for i in prediction[0]:
        sum += i
    confidence = (np.amax(prediction) /  sum) * 100
    
    cv2.putText(textImage,"Gesture: " + predicted_gesture,(x, y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 23, 25),2)
    y += 30
    cv2.putText(textImage,"Confidence: " + str("{:.2f}".format(confidence)) + "%",(x, y),cv2.FONT_HERSHEY_SIMPLEX,1,(23, 255, 25),2)

## Using GPU

In [8]:
#
method1 = False

if method1:
    import os
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    tf.compat.v1.reset_default_graph()
    gpu_options.allow_growth = True
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#
method2 = False

if method2:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 0.7
    tf.compat.v1.keras.backend.set_session(
        tf.compat.v1.Session(config=config))

# 
method3 = False

if method3:
    physical_devices = tf.config.list_physical_devices('GPU')
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

## Loading Model

In [9]:
tf.compat.v1.reset_default_graph()
model = models.load_model('./TrainedModel/')

## Predicting

In [10]:
aWeight = 0.5
top, right, bottom, left = 30, 350, 225, 590
num_frames = 0
start_recording = True

In [11]:
camera = cv2.VideoCapture(0)
while(True):
    grabbed, frame = camera.read()

    if grabbed:
        frame = imutils.resize(frame, width=700)
        frame = cv2.flip(frame, 1)
    
        roi = frame[top:bottom, right:left]

        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            hand = segment(gray)

            if hand is not None:
                (thresholded, segmented) = hand

                cv2.drawContours(frame, [segmented + (right, top)], -1, (0, 0, 255))
                if start_recording:
                    cv2.imwrite(temp_image, thresholded)
                    resizeImage(temp_image)
                    prediction = getPredictedClass()
                    showStatistics(prediction,frame)

                cv2.imshow("Thesholded", thresholded)

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        num_frames += 1
        cv2.imshow("Video Feed", frame)
        keypress = cv2.waitKey(1) & 0xFF
        if keypress == ord("q"):
            break

    else:
        print("Error, Please check your camera")
        print(camera)
        break

# relaease the resources
camera.release()
cv2.destroyAllWindows()


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18780\2905944500.py:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((100, 89), Image.ANTIALIAS)


In [12]:
# remove temporary image file
import os
os.remove(temp_image)